# CLI rough draft (GPT)

To break the program into modules and a main file, we will create several modules to handle different parts of the process:

1. **Database setup and connection**
2. **Fetching stock data using yfinance**
3. **Populating the database with stock data**

Let's break it down into four files: `db_setup.py`, `fetch_data.py`, `populate_db.py`, and `main.py`.

### 1. `db_setup.py`

This module will handle the creation of the database tables.

```python
import mysql.connector

def create_tables(cursor):
    tables = [
        """CREATE TABLE IF NOT EXISTS `Session` (
            `SessionID` INT PRIMARY KEY
        )""",
        """CREATE TABLE IF NOT EXISTS `Portfolio` (
            `PortfolioID` INT PRIMARY KEY,
            `TotalAmt` FLOAT,
            `Risk` VARCHAR(64)
        )""",
        """CREATE TABLE IF NOT EXISTS `Allocation` (
            `AllocID` INT PRIMARY KEY,
            `Ticker` VARCHAR(10),
            `Amount` FLOAT
        )""",
        """CREATE TABLE IF NOT EXISTS `Stock` (
            `StockID` INT PRIMARY KEY AUTO_INCREMENT,
            `Ticker` VARCHAR(10),
            `Sector` VARCHAR(64),
            `Price` FLOAT,
            `SD` FLOAT,
            `ERet` FLOAT
        )""",
        """CREATE TABLE IF NOT EXISTS `History` (
            `HistoryID` INT PRIMARY KEY AUTO_INCREMENT,
            `Ticker` VARCHAR(10),
            `Date` DATE,
            `Price` FLOAT
        )""",
        """CREATE TABLE IF NOT EXISTS `SessionHasPortfolio` (
            `SessionID` INT,
            `PortfolioID` INT,
            FOREIGN KEY (`SessionID`) REFERENCES `Session`(`SessionID`),
            FOREIGN KEY (`PortfolioID`) REFERENCES `Portfolio`(`PortfolioID`),
            PRIMARY KEY (`SessionID`, `PortfolioID`)
        )""",
        """CREATE TABLE IF NOT EXISTS `PortfolioHasAllocation` (
            `PortfolioID` INT,
            `AllocID` INT,
            FOREIGN KEY (`PortfolioID`) REFERENCES `Portfolio`(`PortfolioID`),
            FOREIGN KEY (`AllocID`) REFERENCES `Allocation`(`AllocID`),
            PRIMARY KEY (`PortfolioID`, `AllocID`)
        )""",
        """CREATE TABLE IF NOT EXISTS `PortfolioHasStock` (
            `PortfolioID` INT,
            `StockID` INT,
            FOREIGN KEY (`PortfolioID`) REFERENCES `Portfolio`(`PortfolioID`),
            FOREIGN KEY (`StockID`) REFERENCES `Stock`(`StockID`),
            PRIMARY KEY (`PortfolioID`, `StockID`)
        )""",
        """CREATE TABLE IF NOT EXISTS `AllocationHasStock` (
            `AllocID` INT,
            `StockID` INT,
            FOREIGN KEY (`AllocID`) REFERENCES `Allocation`(`AllocID`),
            FOREIGN KEY (`StockID`) REFERENCES `Stock`(`StockID`),
            PRIMARY KEY (`AllocID`, `StockID`)
        )""",
        """CREATE TABLE IF NOT EXISTS `StockHasHistory` (
            `StockID` INT,
            `HistoryID` INT,
            FOREIGN KEY (`StockID`) REFERENCES `Stock`(`StockID`),
            FOREIGN KEY (`HistoryID`) REFERENCES `History`(`HistoryID`),
            PRIMARY KEY (`StockID`, `HistoryID`)
        )"""
    ]

    for table in tables:
        cursor.execute(table)

def get_db_connection():
    return mysql.connector.connect(
        host="localhost",
        user="yourusername",
        password="yourpassword",
        database="yourdatabase"
    )
```

### 2. `fetch_data.py`

This module will handle fetching the stock data using yfinance.

```python
import yfinance as yf
from datetime import datetime, timedelta

def fetch_stock_data(tickers):
    data = {}
    end_date = datetime.now()
    start_date = end_date - timedelta(days=30)
    
    for ticker in tickers:
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        if not stock_data.empty:
            data[ticker] = stock_data
    return data
```

### 3. `populate_db.py`

This module will handle populating the database with stock data.

```python
def populate_stock_data(cursor, stock_data):
    for ticker, data in stock_data.items():
        last_row = data.iloc[-1]
        cursor.execute(
            "INSERT INTO Stock (Ticker, Price) VALUES (%s, %s)",
            (ticker, last_row['Close'])
        )
        stock_id = cursor.lastrowid

        for date, row in data.iterrows():
            cursor.execute(
                "INSERT INTO History (Ticker, Date, Price) VALUES (%s, %s, %s)",
                (ticker, date.strftime('%Y-%m-%d'), row['Close'])
            )
            history_id = cursor.lastrowid
            cursor.execute(
                "INSERT INTO StockHasHistory (StockID, HistoryID) VALUES (%s, %s)",
                (stock_id, history_id)
            )
```

### 4. `main.py`

This is the main file that will bring everything together.

```python
import db_setup
import fetch_data
import populate_db

def main():
    # Establish database connection
    db = db_setup.get_db_connection()
    cursor = db.cursor()

    # Create tables
    db_setup.create_tables(cursor)

    # Commit table creation
    db.commit()

    # Input tickers from the command line
    tickers = input("Enter the stock tickers separated by commas: ").split(',')

    # Fetch the stock data
    stock_data = fetch_data.fetch_stock_data(tickers)

    # Populate the Stock and History tables
    populate_db.populate_stock_data(cursor, stock_data)

    # Commit the changes
    db.commit()

    # Close the connection
    cursor.close()
    db.close()

if __name__ == "__main__":
    main()
```

### Instructions to Run the Program

1. **Install Dependencies:**
   ```bash
   pip install mysql-connector-python yfinance pandas
   ```

2. **Replace Database Credentials:**
   Replace `yourusername`, `yourpassword`, and `yourdatabase` with your actual MySQL credentials and database name in `db_setup.py`.

3. **Run the Program:**
   Run the main file `main.py`:
   ```bash
   python main.py
   ```

This modular approach makes the code more organized, maintainable, and easier to debug or extend. Each module handles a specific aspect of the process, and `main.py` ties them all together.